In [76]:
import PyPDF2
import re
import os
import openpyxl

In [33]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        pdf = PyPDF2.PdfReader(f)
        text = '\n'.join([pdf.pages[page_num].extract_text() for page_num in range(len(pdf.pages))])
        return text

In [34]:
def get_directory_files(dir_path):
    files = []
    for file_path in os.listdir(dir_path):
        if os.path.isfile(os.path.join(dir_path, file_path)) and (".pdf" in file_path):
            files.append(file_path)

    return files

In [67]:
def get_student_name(file_name):
    name = file_name.split(' ')
    first_name = name[0].split('_')[1]
    last_name = name[1].split('_')[0]
    return first_name, last_name

In [36]:
def all_topological_sorts(graph):
    nodes = list(graph.keys())
    num_nodes = len(nodes)

    in_degree = {node: 0 for node in nodes}

    sorted_list = []
    all_sorts = []  

    visited = {node: False for node in nodes}

    for node in graph:
        for adj in graph[node]:
            in_degree[adj] += 1

    def dfs():
        is_all_visited = True

        for v in nodes:
            if in_degree[v] == 0 and not visited[v]:
                for adj in graph[v]:
                    in_degree[adj] -= 1

                sorted_list.append(v)
                visited[v] = True
                dfs()  


                visited[v] = False
                sorted_list.remove(v)
                for adj in graph[v]:
                    in_degree[adj] += 1
                is_all_visited = False

        if is_all_visited:
            all_sorts.append(sorted_list.copy())  

    dfs()
    return all_sorts  


graph = {
    'A': [],
    'B': ['A', 'C', 'D', 'E'],
    'C': ['D'],
    'D': ['H'],
    'E': [],
    'F': ['D', 'E'],
    'G': ['E', 'F', 'H'],
    'H': []
}


result = all_topological_sorts(graph)
print(len(result))  

132


In [69]:


def generate_regex(char_list):
    pattern = ''
    for char in char_list:
        if char in '.^$*+?{}[]\\|()':
            char = '\\' + char
        pattern += f"(?=.*[{re.escape(char)}])"

    return pattern


In [70]:
def grade(submission, results):
    text = extract_text_from_pdf("/Users/mahmood/Desktop/grading/week11/11/files/" + str(submission))
    first_name, last_name = get_student_name(str(submission))
    count = 0
    for topological_sort in result:

        regex = generate_regex(topological_sort)
        
        match = re.search(regex, text, re.DOTALL)
        if bool(match) == True:
            count += 1
    
    grade = 0
    if count == 132:
        grade = 5
    elif count >= 102 and count < 132:
        grade = 4
    elif count >= 68 and count < 102:
        grade = 3
    elif count >= 34 and count < 68:
        grade = 2
    elif count >= 1 and count < 34:
        grade = 1
    return first_name, last_name, count, grade

In [86]:
def grader(path):
    path_files = get_directory_files(path)
    for file in path_files:
        first_name, last_name, count, score = grade(file, result)
        if first_name is not None and last_name is not None:
            add_to_sheet(first_name, last_name, 1, 2, score)

In [93]:
def add_to_sheet(first_name, last_name, first_name_col, last_name_col, grade):
    file_path = "/Users/mahmood/Desktop/grading/week11/grades.xlsx"
    wb = openpyxl.load_workbook(file_path)
    found_students = []
    print(first_name in "sergeyasdfa")
    for sheet in wb:
        print(f"Searching in sheet: {sheet.title}")
        for row in sheet.iter_rows():

            if len(row) >= max(first_name_col, last_name_col):
                first_name_cell = row[first_name_col - 1]
                last_name_cell = row[last_name_col - 1]
        
                if (first_name in first_name_cell.value) and (last_name in last_name_cell.value):
                    row[4] = grade
                    wb.save(file_path)
                    return


In [94]:
grader('/Users/mahmood/Desktop/grading/week11/11/files')

False
Searching in sheet: 11


TypeError: argument of type 'NoneType' is not iterable